In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [2]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
caffe_root = ""
sys.path.insert(0, caffe_root + 'python')

import caffe
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.
import tempfile

caffe.set_device(0)
caffe.set_mode_gpu()

In [3]:
def run_solvers(niter, solvers, disp_interval=50):
    """Run solvers for niter iterations,
       returning the loss and accuracy recorded each iteration.
       `solvers` is a list of (name, solver) tuples."""
    blobs = ('loss', 'acc')
    loss, acc = ({name: np.zeros(niter) for name, _ in solvers}
                 for _ in blobs)
    for it in range(niter):
        for name, s in solvers:
            s.step(1)  # run a single SGD step in Caffe
            loss[name][it], acc[name][it] = (s.net.blobs[b].data.copy()
                                             for b in blobs)
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = '; '.join('%s: loss=%.3f, acc=%2d%%' %
                                  (n, loss[n][it], np.round(100*acc[n][it]))
                                  for n, _ in solvers)
            print '%3d) %s' % (it, loss_disp)     
    # Save the learned weights from both nets.
#     weight_dir = tempfile.mkdtemp()
    weights = {}
#     for name, s in solvers:
#         filename = 'weights.%s.caffemodel' % name
#         weights[name] = os.path.join(weight_dir, filename)
#         s.net.save(weights[name])
    return loss, acc, weights

In [4]:
def eval_val_accuracy(test_net, test_iters=10):
    accuracy = 0
    for it in xrange(test_iters):
        accuracy += test_net.forward()['acc']
    accuracy /= test_iters
    return test_net, accuracy

In [5]:
model_path = caffe_root + "models/deep_mammo/cut1_train_val.prototxt"
pretrained_weights_path = caffe_root + "models/deep_mammo/VGG_ILSVRC_16_layers.caffemodel"
quick_solver_path = caffe_root + "models/deep_mammo/quick_solver.prototxt"
# net = caffe.Net(model_path, pretrained_weights_path, caffe.TRAIN)

In [6]:
niter = 200  # number of iterations to train

quick_solver = caffe.get_solver(quick_solver_path)
quick_solver.net.copy_from(pretrained_weights_path)

print 'Running solvers for %d iterations...' % niter
solvers = [('pretrained', quick_solver)]
loss, acc, weights = run_solvers(niter, solvers)
print 'Done.'

train_loss = loss['pretrained']
train_acc = acc['pretrained']

# Delete solvers to save memory.
del quick_solver, solvers

Running solvers for 200 iterations...
  0) pretrained: loss=9.952, acc=62%
 40) pretrained: loss=16.851, acc=50%
 80) pretrained: loss=0.680, acc=69%
120) pretrained: loss=0.695, acc=50%
160) pretrained: loss=0.694, acc=38%
199) pretrained: loss=0.745, acc=25%
Done.


KeyError: 'pretrained'

In [ ]:
plot(train_loss.T)
xlabel('Iteration #')
ylabel('Loss')

In [ ]:
plot(train_acc.T)
xlabel('Iteration #')
ylabel('Accuracy')